In [1]:
from outlines.models import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import transformers
from torch.nn.attention.flex_attention import create_block_mask, and_masks, or_masks, create_mask


/venv/goedelv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_anneal_attn_mask(seq_len, bsz, dtype, device, attn_mask_ratio):
    mask = torch.full((seq_len, seq_len), 0, device=device)
    mask_cond = torch.arange(mask.size(-1), device=device)
    mask.masked_fill_(mask_cond < (mask_cond + 1).view(mask.size(-1), 1), 1)
    causal_mask = mask.to(dtype)

    random_mask = torch.bernoulli(torch.full((seq_len, seq_len), 0.0, device=device) + attn_mask_ratio)

    anneal_mask = torch.logical_or(causal_mask, random_mask)
    expanded_mask = anneal_mask[None, None, :, :].expand(bsz, 1, seq_len, seq_len)
    inverted_mask = 1.0 - expanded_mask.to(dtype)

    return inverted_mask.masked_fill(inverted_mask.to(torch.bool), torch.finfo(dtype).min)
    return anneal_mask



In [3]:
# initial mask is bsz x seq_len, want bsz x seq_len x seq_len
def get_anneal_mask_new(mask, attn_mask_ratio):

    random_mask = torch.bernoulli(torch.full((seq_len, seq_len), 0.0, device=device) + attn_mask_ratio)

In [4]:
# looks like Qwen using SDPA should be fine with an adapted attention mask, no


from functools import partial
from transformers import DynamicCache
from transformers.processing_utils import Unpack
from transformers.modeling_outputs import BaseModelOutputWithPast
from transformers.modeling_flash_attention_utils import FlashAttentionKwargs
from transformers.masking_utils import create_causal_mask, create_sliding_window_causal_mask
from cachetools import Cache
from typing import Optional, __all__
from transformers.utils import logging, TransformersKwargs

logger = logging.get_logger(__name__)


# def qwen_new_forward(
#         self,
#         input_ids: Optional[torch.LongTensor] = None,
#         attention_mask: Optional[torch.Tensor] = None,
#         position_ids: Optional[torch.LongTensor] = None,
#         past_key_values: Optional[Cache] = None,
#         inputs_embeds: Optional[torch.FloatTensor] = None,
#         use_cache: Optional[bool] = None,
#         output_attentions: Optional[bool] = None,
#         output_hidden_states: Optional[bool] = None,
#         cache_position: Optional[torch.LongTensor] = None,
#         **flash_attn_kwargs: Unpack[FlashAttentionKwargs],
# ) -> BaseModelOutputWithPast:
#
#     output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
#     output_hidden_states = (
#         output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
#     )
#     use_cache = use_cache if use_cache is not None else self.config.use_cache
#
#     if (input_ids is None) ^ (inputs_embeds is not None):
#         raise ValueError("You must specify exactly one of input_ids or inputs_embeds")
#
#     if self.gradient_checkpointing and self.training and use_cache:
#         logger.warning_once(
#             "`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`."
#         )
#         use_cache = False
#
#     # TODO (joao): remove this exception in v4.56 -- it exists for users that try to pass a legacy cache
#     if not isinstance(past_key_values, (type(None), Cache)):
#         raise ValueError("The `past_key_values` should be either a `Cache` object or `None`.")
#
#     if inputs_embeds is None:
#         inputs_embeds = self.embed_tokens(input_ids)
#
#     if use_cache and past_key_values is None:
#         past_key_values = DynamicCache()
#
#     if cache_position is None:
#         past_seen_tokens = past_key_values.get_seq_length() if past_key_values is not None else 0
#         cache_position = torch.arange(
#             past_seen_tokens, past_seen_tokens + inputs_embeds.shape[1], device=inputs_embeds.device
#         )
#
#     if position_ids is None:
#         position_ids = cache_position.unsqueeze(0)
#
#     ## Add by DiffuLLaMA, adapting for 4d attention-mask.
#     if attention_mask is not None and len(attention_mask.shape) == 4:
#         causal_mask = attention_mask
#         print("logging....attention-mask for 4d")
#     else:
#         causal_mask = self._update_causal_mask(
#             attention_mask, inputs_embeds, cache_position, past_key_values, output_attentions
#         )
#
#     print(causal_mask)
#
#     # causal_mask = self._update_causal_mask(
#     #     attention_mask, inputs_embeds, cache_position, past_key_values, output_attentions
#     # )
#
#     hidden_states = inputs_embeds
#
#     # create position embeddings to be shared across the decoder layers
#     position_embeddings = self.rotary_emb(hidden_states, position_ids)
#
#     # decoder layers
#     all_hidden_states = () if output_hidden_states else None
#     all_self_attns = () if output_attentions else None
#
#     for decoder_layer in self.layers[: self.config.num_hidden_layers]:
#         if output_hidden_states:
#             all_hidden_states += (hidden_states,)
#
#         if self.gradient_checkpointing and self.training:
#             layer_outputs = self._gradient_checkpointing_func(
#                 partial(decoder_layer.__call__, **flash_attn_kwargs),
#                 hidden_states,
#                 causal_mask,
#                 position_ids,
#                 past_key_values,
#                 output_attentions,
#                 use_cache,
#                 cache_position,
#                 position_embeddings,
#             )
#         else:
#             layer_outputs = decoder_layer(
#                 hidden_states,
#                 attention_mask=causal_mask,
#                 position_ids=position_ids,
#                 past_key_value=past_key_values,
#                 output_attentions=output_attentions,
#                 use_cache=use_cache,
#                 cache_position=cache_position,
#                 position_embeddings=position_embeddings,
#                 **flash_attn_kwargs,
#             )
#
#         hidden_states = layer_outputs[0]
#
#         if output_attentions:
#             all_self_attns += (layer_outputs[1],)
#
#     hidden_states = self.norm(hidden_states)
#
#     # add hidden states from the last decoder layer
#     if output_hidden_states:
#         all_hidden_states += (hidden_states,)
#
#     return BaseModelOutputWithPast(
#         last_hidden_state=hidden_states,
#         past_key_values=past_key_values if use_cache else None,
#         hidden_states=all_hidden_states,
#         attentions=all_self_attns,
#     )
#

In [5]:
def qwen_new_forward(
    self,
    input_ids: Optional[torch.LongTensor] = None,
    attention_mask: Optional[torch.Tensor] = None,
    position_ids: Optional[torch.LongTensor] = None,
    past_key_values: Optional[Cache] = None,
    inputs_embeds: Optional[torch.FloatTensor] = None,
    use_cache: Optional[bool] = None,
    cache_position: Optional[torch.LongTensor] = None,
    **kwargs: Unpack[TransformersKwargs],
) -> BaseModelOutputWithPast:
    if (input_ids is None) ^ (inputs_embeds is not None):
        raise ValueError("You must specify exactly one of input_ids or inputs_embeds")

    if inputs_embeds is None:
        inputs_embeds = self.embed_tokens(input_ids)

    if use_cache and past_key_values is None:
        past_key_values = DynamicCache(config=self.config)

    if cache_position is None:
        past_seen_tokens = past_key_values.get_seq_length() if past_key_values is not None else 0
        cache_position = torch.arange(
            past_seen_tokens, past_seen_tokens + inputs_embeds.shape[1], device=inputs_embeds.device
        )

    if position_ids is None:
        position_ids = cache_position.unsqueeze(0)

    # It may already have been prepared by e.g. `generate`

    # if not isinstance(causal_mask_mapping := attention_mask, dict):

    print (self.has_sliding_layers)
    print (self.config._attn_implementation)

    # if len(attention_mask.shape) == 4:
    #     causal_mask_mapping = {'full_attention': attention_mask}
    #     print (attention_mask)
    #
    if not isinstance(causal_mask_mapping := attention_mask, dict):

        # Prepare mask arguments
        mask_kwargs = {
            "config": self.config,
            "input_embeds": inputs_embeds,
            "attention_mask": attention_mask,
            "cache_position": cache_position,
            "past_key_values": past_key_values,
            "position_ids": position_ids,
        }
        # Create the masks
        causal_mask_mapping = {
            "full_attention": create_causal_mask(**mask_kwargs),
        }
        # The sliding window alternating layers are not always activated depending on the config
        if self.has_sliding_layers:
            causal_mask_mapping["sliding_attention"] = create_sliding_window_causal_mask(**mask_kwargs)

    print (causal_mask_mapping.items())

    hidden_states = inputs_embeds

    # create position embeddings to be shared across the decoder layers
    position_embeddings = self.rotary_emb(hidden_states, position_ids)

    for decoder_layer in self.layers[: self.config.num_hidden_layers]:
        hidden_states = decoder_layer(
            hidden_states,
            attention_mask=causal_mask_mapping[decoder_layer.attention_type],
            position_ids=position_ids,
            past_key_values=past_key_values,
            use_cache=use_cache,
            cache_position=cache_position,
            position_embeddings=position_embeddings,
            **kwargs,
        )

    hidden_states = self.norm(hidden_states)
    return BaseModelOutputWithPast(
        last_hidden_state=hidden_states,
            past_key_values=past_key_values if use_cache else None,
        )


In [6]:
# transformers.models.qwen3.modeling_qwen3.Qwen3Model.forward = qwen_new_forward

In [7]:
model_id = "Goedel-LM/Goedel-Prover-V2-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, dtype=torch.bfloat16, trust_remote_code=True, _attn_implementation='flex_attention')


Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 110.97it/s]


In [8]:
formal_statement = """
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat


theorem square_equation_solution {x y : ℝ} (h : x^2 + y^2 = 2*x - 4*y - 5) : x + y = -1 := by
  sorry
""".strip()

prompt = """
Complete the following Lean 4 code:

```lean4
{}```

Before producing the Lean 4 code to formally prove the given theorem, provide a detailed proof plan outlining the main proof steps and strategies.
The plan should highlight key ideas, intermediate lemmas, and proof structures that will guide the construction of the final formal proof.
""".strip()

chat = [
    [{"role": "user", "content": 'ABC' * 70}],
    [{'role': 'user', 'content': 'ABCDEF' * 70}]
]

inputs = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, padding='longest',
                                       return_dict=True, return_tensors='pt')

In [9]:
inputs

{'input_ids': tensor([[151644,    872,    198,  25411,  25411,  25411,  25411,  25411,  25411,
          25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,
          25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,
          25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,
          25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,
          25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,
          25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,
          25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,
          25411, 151645,    198, 151644,  77091,    198, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         15164

In [10]:
inputs['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
      

In [11]:
def causal_mask(b,h,q_idx,kv_idx):
    return q_idx >= kv_idx

def create_padding_mask(pads):
    def padding(b, h, q_idx, kv_idx):
        return ~pads[b, q_idx] & ~pads[b, kv_idx]
    return padding


attn_mask_ratio = 0.5
seq_len = inputs['input_ids'].shape[1]

random_mask = (torch.rand(seq_len, seq_len) < attn_mask_ratio).to('cuda')
pads = inputs['attention_mask'].to('cuda')
padding_mask = create_padding_mask(pads)

def create_random_mask(b,h,q_idx,kv_idx):
    return random_mask[q_idx][kv_idx]


# precompute block mask for after attention annealing, for attention annealing implement in forward method

# full_mask = and_masks(*[or_masks(*[causal_mask, create_random_mask]), padding_mask])
full_mask = or_masks(*[causal_mask, create_random_mask])


# block_mask = create_block_mask(full_mask, inputs['input_ids'].shape[0], None, len(inputs['input_ids'][0]), len(inputs['input_ids'][0]), device='cuda')
block_mask = create_block_mask(causal_mask, None, None, len(inputs['input_ids'][0]), len(inputs['input_ids'][0]), device='cuda', _compile=True)



block_mask_2 = create_block_mask(causal_mask, None, None, len(inputs['input_ids'][0]), len(inputs['input_ids'][0]), device='cuda')


In [12]:
test_mask = create_mask(full_mask, None, None, len(inputs['input_ids'][0]), len(inputs['input_ids'][0]), device='cuda')


In [13]:
test_mask

tensor([[[[ True, False,  True,  ..., False,  True, False],
          [ True,  True, False,  ..., False, False,  True],
          [ True,  True,  True,  ..., False, False,  True],
          ...,
          [ True,  True,  True,  ...,  True, False, False],
          [ True,  True,  True,  ...,  True,  True,  True],
          [ True,  True,  True,  ...,  True,  True,  True]]]], device='cuda:0')

In [14]:
block_mask


BlockMask(
    kv_num_blocks=torch.Size([1, 1, 2]),
    kv_indices=torch.Size([1, 1, 2, 2]),
    full_kv_num_blocks=torch.Size([1, 1, 2]),
    full_kv_indices=torch.Size([1, 1, 2, 2]),
    q_num_blocks=torch.Size([1, 1, 2]),
    q_indices=torch.Size([1, 1, 2, 2]),
    full_q_num_blocks=torch.Size([1, 1, 2]),
    full_q_indices=torch.Size([1, 1, 2, 2]),
    BLOCK_SIZE=(128, 128),
    shape=(1, 1, 148, 148),
    sparsity=-124.40%,
    mask_mod=causal_mask
)

In [15]:
print (block_mask_2.mask_mod)

<function causal_mask at 0x7f7e997b0dc0>


In [16]:
block_mask

BlockMask(
    kv_num_blocks=torch.Size([1, 1, 2]),
    kv_indices=torch.Size([1, 1, 2, 2]),
    full_kv_num_blocks=torch.Size([1, 1, 2]),
    full_kv_indices=torch.Size([1, 1, 2, 2]),
    q_num_blocks=torch.Size([1, 1, 2]),
    q_indices=torch.Size([1, 1, 2, 2]),
    full_q_num_blocks=torch.Size([1, 1, 2]),
    full_q_indices=torch.Size([1, 1, 2, 2]),
    BLOCK_SIZE=(128, 128),
    shape=(1, 1, 148, 148),
    sparsity=-124.40%,
    mask_mod=causal_mask
)

In [17]:
inputs['input_ids'].shape

torch.Size([2, 148])

In [18]:
model.cuda()

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 4096)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((4096,), eps=1e-06)
        (post_attention_la

In [19]:
# torch.compile(model)

In [20]:
print (inputs)

{'input_ids': tensor([[151644,    872,    198,  25411,  25411,  25411,  25411,  25411,  25411,
          25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,
          25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,
          25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,
          25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,
          25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,
          25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,
          25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,  25411,
          25411, 151645,    198, 151644,  77091,    198, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         15164

In [21]:
out = model.forward(inputs['input_ids'].to('cuda'), attention_mask=block_mask, output_hidden_states=True, output_attentions=True)


InductorError: RuntimeError: No valid triton configs. OutOfMemoryError: out of resource: triton_tem_fused_0 Required: 107008 Hardware limit:101376 Reducing block sizes or `num_stages` may help.

Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (please do this especially if you're reporting a bug to PyTorch). For even more developer context, set TORCH_LOGS="+dynamo"


In [ ]:
out.attentions[0][0][2]

In [ ]:
# assert attentions are 0 at attn_mask, and nonzero elsewhere


# new_mask = attn_mask.expand(-1, logits.attentions[0].shape[1], -1,-1)
#
# for layer in logits.attentions:
#     for x_ in layer:
#         for attns in x_:
#             print (attns)
#             break

# comparing to attn_mask, looks like the attentions are correctly working. I.e. attentions are 0 for masked, and non-zero elsewhere
# Interestingly, high attention values often found for subsequent tokens even though model only used to seeing past tokens.
# Suggests annealing worthwhile to limit influence earlier in adaptation process

In [ ]:
# todo:
# - set up dataset, test edit flow data prep code (couplings, alignments etc. ).
# - Look at generating from empty coupling as well as error correcting coupling (i.e. add error to context, modify the code directly)
# - Set up model wrapper (rate predictions etc.), Quantization/LoRA. Unlike MDM adaptation, we can probably keep the logits unshifted, as we are predicting both substitute and insert next token probs, as well as delete.
